# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     ----------- -------------------------- 307.2/990.0 kB 6.3 MB/s eta 0:00:01
     ----------------------- -------------- 604.2/990.0 kB 6.3 MB/s eta 0:00:01
     -------------------------------------- 990.0/990.0 kB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/373.5 kB ? eta -:--:--
     ------------------------------------- 373.5/373.5 kB 11.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
     -------------------------------------- 139.8/139.8 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.65
    Uninstalling langsmith-0.1.65:
      Successfully uninstalled langsmith-0.1.65
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      Successfully un

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [7]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [37]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位很棒的生物学家。你擅长回答生物问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


computer_template = """你是一位很棒的程序员。你擅长回答计算机相关的问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

chinese_language_template = """你是一位很棒的汉语文学家。你擅长回答汉语文问题。
你会自由而富有诗意地表达自己的想法

这是一个问题：
{input}"""

In [38]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语文",
        "description": "适用于回答汉语文问题",
        "prompt_template": chinese_language_template,
    },
]

In [39]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [40]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [41]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [42]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [43]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [44]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语文: 适用于回答汉语文问题']


In [45]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语文: 适用于回答汉语文问题


In [46]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [47]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语文: 适用于回答汉语文问题

<< INPUT >>
{input}

<< OU

In [48]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [49]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...


APITimeoutError: Request timed out.

In [50]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...


APITimeoutError: Request timed out.

In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极端密度和强大引力的天体，它的重力场非常强大，甚至连光也无法逃脱。它的存在是由于恒星在死亡时发生了坍缩，使得它们的质量集中在极小的空间内。黑洞的存在已经被多项观测和理论证据所证实，但仍然有很多未解之谜。'}


In [53]:
print(chain.invoke("什么是生殖隔离？"))



> Entering new MultiPromptChain chain...


APITimeoutError: Request timed out.

In [52]:
print(chain.invoke("HTTPS为什么比THHP安全？"))



> Entering new MultiPromptChain chain...


APITimeoutError: Request timed out.

In [51]:
print(chain.invoke("巴山楚水凄凉地，如何理解？"))



> Entering new MultiPromptChain chain...


APITimeoutError: Request timed out.

### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains